 ### Loading and importing all the libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Function to draw a rectangle using four corners provided in `biggestNew` on `mainFrame`
def drawRec(biggestNew, mainFrame):
    """
    Draws a rectangle using four corners provided in `biggestNew` on `mainFrame`.

    Parameters:
    - biggestNew: numpy array of shape (4,1,2) containing the coordinates of the corners.
    - mainFrame: the image on which to draw the rectangle.
    """
    # Draw lines between each pair of corners to form a rectangle
    cv2.line(mainFrame, tuple(biggestNew[0][0]), tuple(biggestNew[1][0]), (0, 255, 0), 20)
    cv2.line(mainFrame, tuple(biggestNew[0][0]), tuple(biggestNew[2][0]), (0, 255, 0), 20)
    cv2.line(mainFrame, tuple(biggestNew[3][0]), tuple(biggestNew[2][0]), (0, 255, 0), 20)
    cv2.line(mainFrame, tuple(biggestNew[3][0]), tuple(biggestNew[1][0]), (0, 255, 0), 20)

### Load and resize the image

In [ ]:
img = cv2.imread("insert your image here") # Replace "your_image_path" with the path to your image
img = cv2.resize(img, (int(480*2), int(640*2)))  # Resize for processing
w, h = 480, 640  # Target width and height for perspective transformation
imgWarp = img.copy()  # Create a copy of the original image for warping

### Preprocessing: Convert to grayscale, apply Gaussian blur, and detect edges using Canny

In [ ]:
GrayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
BlurredFrame = cv2.GaussianBlur(GrayImg, (5, 5), 1)
CannyFrame = cv2.Canny(BlurredFrame, 190, 190)


### Find contours in the edge-detected image

In [ ]:
contours, _ = cv2.findContours(CannyFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

### Draw contours on a copy of the original image

In [ ]:
ContourFrame = img.copy()
ContourFrame = cv2.drawContours(ContourFrame, contours, -1, (255, 0, 255), 4)


### Initialize variables for finding the largest quadrilateral (document corners)

In [ ]:
maxArea = 0
biggest = []

### Iterate through each contour to find the largest quadrilateral

In [ ]:
for i in contours:
    area = cv2.contourArea(i)
    if area > 500:  # Only consider contours with area greater than 500
        peri = cv2.arcLength(i, True)  # Calculate the perimeter of the contour
        edges = cv2.approxPolyDP(i, 0.02*peri, True)  # Approximate the contour with fewer vertices
        if area > maxArea and len(edges) == 4:  # Select the largest quadrilateral
            biggest = edges
            maxArea = area

# If a quadrilateral is found, proceed with perspective transformation
if len(biggest) != 0:
    biggest = biggest.reshape((4, 2))  # Reshape to a 4x2 array of corner coordinates
    biggestNew = np.zeros((4, 1, 2), dtype=np.int32)

    # Order the corners clockwise starting from the top-left corner
    add = biggest.sum(1)
    biggestNew[0] = biggest[np.argmin(add)]
    biggestNew[3] = biggest[np.argmax(add)]
    dif = np.diff(biggest, axis=1)
    biggestNew[1] = biggest[np.argmin(dif)]
    biggestNew[2] = biggest[np.argmax(dif)]

    # Draw the quadrilateral and apply perspective transformation
    drawRec(biggestNew, img)  # Draw the rectangle on the original image
    CornerFrame = cv2.drawContours(img.copy(), [biggestNew], -1, (255, 0, 255), 25)  # Draw corners on a separate frame

    # Perspective transformation to warp the document to a rectangular shape
    pts1 = np.float32(biggestNew)
    pts2 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (w, h))

# Resize images for display using matplotlib
img = cv2.resize(img, (480, 640))
GrayImg = cv2.resize(GrayImg, (480, 640))
BlurredFrame = cv2.resize(BlurredFrame, (480, 640))
CannyFrame = cv2.resize(CannyFrame, (480, 640))
ContourFrame = cv2.resize(ContourFrame, (480, 640))
CornerFrame = cv2.resize(CornerFrame, (480, 640))

### Display images using matplotlib for persistent viewing

In [ ]:
def display_image(title, image):
    """Display an image using matplotlib."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.title(title)
    plt.imshow(image_rgb)
    plt.axis('off')

### Display each image with appropriate titles


In [ ]:
display_image("Original Image", img)
display_image("Grayscale Image", GrayImg)
display_image("Blurred Image", BlurredFrame)
display_image("Canny Edge Detection", CannyFrame)
display_image("Contours Detected", ContourFrame)
display_image("Document Corners Detected", CornerFrame)
display_image("Warped Perspective", imgWarp)

plt.show()  # Show all plotted images